# Градиентный бустинг. Имплементации

В sklearn есть наивная версия версия градиентного бустинга, [придуманная](https://projecteuclid.org/download/pdf_1/euclid.aos/1013203451) в 1999 году Фридманом. С тех пор было предложено много реализаций, которые оказываются лучше на практике. На сегодняшний день популярны три библиотеки, реализующие градиентный бустинг:
* XGBoost. После выхода быстро набрала популярность и оставалась стандартом до конца 2016 года.
* LightGBM. Отличительной чертой является быстрота построения композиции. Например, используется следующий трюк для ускорения обучения: при построении вершины дерева вместо перебора по всем значениям признака производится перебор значений гистограммы этого признака. Таким образом, вместо $O(\ell)$ требуется $O$(#bins). Кроме того, в отличие от других библиотек, которые строят дерево по уровням, LightGBM использует стратегию best-first, т.е. на каждом шаге строит вершину, дающую наибольшее уменьшение функционала. Таким образом каждое дерево является цепочкой с прикрепленными листьями, поэтому ограничение на num_leaves получается более осмысленным.
* CatBoost. Библиотека от компании Яндекс. Позволяет автоматически обрабатывать категориальные признаки. Кроме того, алгоритм является менее чувствительным к выбору конкретных гиперпараметров. За счёт этого уменьшается время, которое тратит человек на подбор оптимальных гиперпараметров.

In [ ]:
import xgboost as xgb
import lightgbm as gbm
import catboost

In [ ]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
data = pd.read_csv("flights.csv")
data = data.sample(frac=0.1, random_state=10)

### Задача 1

1) Оставьте колонки "MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"
                 
2) Удалите пропуски в датасете

3) Создайте столбец  target с 1 и 0: 1, если опоздание более чем на 10 минут, 0 - меньше или ровно 10 минут

4) Для листа cols закодируйте колонки с категориальными переменными в числа (т.е. текстовым переменным поставим в соответствие числа

5) Разбейте данные на тренировочную и тестовую выборку в соотношении 0.75, 0.25

In [ ]:
cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]


'''Your code here.'''

### Задача 2

Создайте пользовательскую функцию print_auc с тремя параметрами:
- model: модель, которая будет делать прогнозы
- train: тренировочные данные
- test: тестовые данные

Функция возвращает два значения: roc_auc для тренировочной выборки и roc_auc для тестовой выборки

In [ ]:
'''Your code here.'''


Все бустинговые библиотеки поддерживают формат обучения моделей, как в sklearn, так и свои собственные элементы, поэтому помимо основой части кода с датасетом про полеты мы еще будем смотреть, как работать в логике самой библиотеки

## XGBoost

### Cчитывание данных

Вот так, например, можно задать данные, чтобы считывание проходило быстрее:

In [ ]:
dtrain = xgb.DMatrix(train, label=y_train)

Но данные у нас уже записаны и можем воспользоваться классическим подходом по типу sklearn

### Обучение модели

Как правило, параметры задают отдельно в словаре:

In [ ]:
params_xgb = {"max_depth": 10,
             "min_child_weight" : 2,
              "n_estimators": 200,
              "learning_rate": 0.1}

In [ ]:
bst = xgb.train(params_xgb, dtrain)

### Задание 3
Но можно и обучить в стиле sklearn: 

- Объявите модель xgb.XGBClassifier с параметрами выше
- Обучите модель

In [ ]:
'''Your code here.'''


### Прогноз

Прогнозы получаются стандартно - методом predict.

### Сохранение модели

In [ ]:
bst.dump_model('dump.raw.txt')

### Считывание модели

In [ ]:
bst = xgb.Booster()  # init model
bst.load_model('model.bin')  # load data

## LightGBM/ Catboost

### Основные параметры

(lightgbm/catboost)

* objective – функционал, на который будет настраиваться композиция
* eta / learning_rate – темп (скорость) обучения
* num_iterations / n_estimators  – число итераций бустинга

#### Параметры, отвечающие за сложность деревьев
* max_depth – максимальная глубина 
* max_leaves / num_leaves – максимальное число вершин в дереве
* gamma / min_gain_to_split – порог на уменьшение функции ошибки при расщеплении в дереве
* min_data_in_leaf – минимальное число объектов в листе
* min_sum_hessian_in_leaf – минимальная сумма весов объектов в листе, минимальное число объектов, при котором делается расщепление 
* lambda – коэффициент регуляризации (L2)
* subsample / bagging_fraction – какую часть объектов обучения использовать для построения одного дерева 
* colsample_bytree / feature_fraction – какую часть признаков использовать для построения одного дерева 

Подбор всех этих параметров — настоящее искусство. Но начать их настройку можно с самых главных параметров: learning_rate и n_estimators. Обычно один из них фиксируют, а оставшийся из этих двух параметров подбирают (например, фиксируют n_estimators=1000 и подбирают learning_rate). Следующим по важности является max_depth. В силу того, что мы заинтересованы в неглубоких деревьях, обычно его перебирают из диапазона [3; 7].




## LightGBM

In [ ]:
params_gbm = {
    "objective": 'binary',
    "max_depth": 10,
    "min_child_weight" : 2,
    "n_estimators": 200,
    "learning_rate": 0.1}

In [ ]:
d_train = lgb.Dataset(train, label=y_train)

# Without Categorical Features
model2 = lgb.train(params_gbm, d_train)

In [ ]:
#With Catgeorical Features
cats_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT"]
model2 = lgb.train(params, d_train, categorical_feature=cats_features_name)

In [ ]:
model2.booster_.save_model('lightgbm.txt')

## CatBoost

In [ ]:
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train, y_train)

In [ ]:
cat_features_index = [0,1,2,3,4,5,6]

# With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
                            depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)

clf.fit(train,y_train, cat_features=cat_features_index)

In [ ]:
clf.save_model('catboost.cbm', format='cbm')
clf = clf.load_model('catboost.cbm')

## Образец гридсерча

In [ ]:
lg = lgb.LGBMClassifier(silent=False)
param_dist = {"max_depth": [25,50, 75],
              "learning_rate" : [0.01,0.05,0.1],
              "num_leaves": [300,900,1200],
              "n_estimators": [200]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(train, y_train)
grid_search.best_estimator_

## LightGBM боевого образца
(пишем вместе)

In [ ]:
'''Your code here.'''


# Стэкинг моделей 

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

def rmse(y_true, y_pred):
    error = (y_true - y_pred) ** 2
    return np.sqrt(np.mean(error))

### Задача 4

- Напишите функцию rmse, рассчитывающую rmse для прогнозных и фактических значениях

In [ ]:
'''Your code here.'''


### Задача 5

- Обучите CatBoostRegressor
- Сохраните прогнозы для тестовой выборки в переменной y_pred_cbm
- Сохраните прогнозы для тренировочной выборки в переменной y_train_pred_cbm

In [ ]:
from catboost import CatBoostRegressor

cbm = CatBoostRegressor(iterations=100, max_depth=4, learning_rate=0.01, loss_function='RMSE', logging_level='Silent')


'''Your code here.'''


print("Train RMSE GB = %.4f" % rmse(y_train, y_train_pred_cbm))
print("Test RMSE GB = %.4f" % rmse(y_test, y_pred_cbm))

### Задача 6

- Отшкалируйте данные
- Обучите линейную регрессию
- Сохраните прогнозы для тестовой выборки в переменной y_pred_lr
- Сохраните прогнозы для тренировочной выборки в переменной y_train_pred_lr

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

'''Your code here.
'''

print("Train RMSE LR = %.4f" % rmse(y_train, y_train_pred_lr))
print("Test RMSE LR = %.4f" % rmse(y_test, y_pred_lr))

Для простоты будем считать, что новый алгоритм $a(x)$ представим как
$$
    a(x)
    =
    \sum_{n = 1}^{N}
    w_n b_n(x),
$$
где $\sum_{n} w_n =1$

In [ ]:
def select_weights(y_true, y_pred_1, y_pred_2):
    grid = np.linspace(0, 1, 1000)
    metric = []
    for w_0 in grid:
        w_1 = 1 - w_0
        y_a = w_0 * y_pred_1 + w_1 * y_pred_2
        metric.append([rmse(y_true, y_a), w_0, w_1])
    return metric

In [ ]:
rmse_blending_train, w_0, w_1 = min(select_weights(y_train, y_train_pred_cbm, y_train_pred_lr), key=lambda x: x[0])

In [ ]:
rmse_blending_train

In [ ]:
rmse(y_test, y_pred_cbm * w_0 +  y_pred_lr * w_1)

В итоге получаем качество на тестовой выборке лучше, чем у каждого алгоритма в отдельности.

Полезные ссылки:


* [Документация XGBoost](https://xgboost.readthedocs.io/en/latest/python/python_intro.html)
* [Документация LightGBM](https://lightgbm.readthedocs.io/en/latest/Python-Intro.html)
* [Документация CatBoost](https://catboost.ai/docs/concepts/python-quickstart.html)
* [Видео про стекинг](https://www.coursera.org/lecture/competitive-data-science/stacking-Qdtt6)
* [Продвинутый вариант стекинга с использованием нейросетей](https://www.coursera.org/learn/competitive-data-science/lecture/s8RLi/stacknet)